# Pre-process our tennis data set so we can feed in the data to ML and DL model

We will do the following in his notebook:

## Reading Data
* data from this dataset is split out by year - we will read in all match data from 1998 (Federer turns pro) to Austrialian Open 2019 (2019/1/14)

## Clean Data
* rename tourney_id because this is misleading - this is a composite of year and tourney id
* clean string columns - remove special characters and accents
* filter out tournments before 1998 (when Federer went pro)
* filter out non-professional tournaments (ie, challenger, satellites)
* clean out tiebreak scores from scores since we will not be using this as part of our features

## Impute Data
* get match minutes from average minutes for the tournament
* player height from average player height
* draw_size from size of tournament
* player rank from average player rank in the tournament - winner rank will be derived from winners, lower rank will be derived from losers


At the end of this notebook, we are left with around 59k entries. The only missing data we have are match stats which we won't be using for the time being


Match data source: https://github.com/JeffSackmann/tennis_atp

In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline



# Reading Dataset

In [28]:
DATASET_DIR = '../datasets'
START_YEAR = 1998
END_YEAR = 2019
OUTFILE = f'{DATASET_DIR}/atp_matches_preprocessed.csv'
years = np.arange(1998, END_YEAR + 1)

matches_orig = pd.DataFrame()
for year in years:
    matches_orig = matches_orig.append(pd.read_csv(f'{DATASET_DIR}/tennis_atp-master/atp_matches_{year}.csv', parse_dates=["tourney_date"]), ignore_index=True)
    
    
print(len(matches_orig))
# read in our ATP data set and parse tournament dates
# matches_orig = pd.read_csv(f'{DATASET_DIR}/tennis_atp-master/ATP.csv', parse_dates=["tourney_date"])
matches_orig.sample(10).T

66348


,36837,9274,43968,28900,35459,41339,64439,53588,7883,50955
tourney_id,2009-D006,2000-421,2011-D039,2006-418,2008-560,2010-421,2018-0311,2014-D075,2000-D009,2013-429
tourney_name,Davis Cup WG R1: ROU vs RUS,Canada Masters,Davis Cup WG QF: ARG vs KAZ,Washington,US Open,Canada Masters,London,Davis Cup G2 R3: DEN vs MDA,Davis Cup G1 SF: UZB vs NZL,Stockholm
surface,Carpet,Hard,Clay,Hard,Hard,Hard,Grass,NaN,Hard,Hard
draw_size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tourney_level,D,M,D,A,G,M,A,D,D,A
tourney_date,2009-03-06 00:00:00,2000-07-31 00:00:00,2011-07-07 00:00:00,2006-07-31 00:00:00,2008-08-25 00:00:00,2010-08-09 00:00:00,2018-06-18 00:00:00,2014-09-12 00:00:00,2000-04-07 00:00:00,2013-10-14 00:00:00
match_num,1,61,1,46,42,19,273,5,5,5
winner_id,103498,103248,104338,104918,103498,104926,103898,104952,103439,104997
winner_seed,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN
winner_entry,NaN,Q,NaN,NaN,NaN,Q,Q,NaN,NaN,NaN


# Data Cleaning

In [3]:
# make all column names lower case so it's easier to remember
matches = matches_orig
matches.columns = [col.strip().lower() for col in matches.columns]

# these columns don't have much data from our EDA so we can't impute. Let's drop them
# for rank points - these are used to determine a player's ranking in the ATP at the moment of the tournament so it's duplicate of rank - will drop this as well
drop_columns = ["loser_entry", "winner_entry", "loser_rank_points", "winner_rank_points"]
matches = matches.drop(drop_columns, axis=1)


# we only care about professional tournaments since we are prediction grand slams so let's filter out non-professional tournaments
# ATP was formed in 1972. Federer turned Pro in 1998, we will only look at matches since January 1998
# We
matches = matches[(~matches.tourney_level.isin(["C", "S", "D"])) & (matches.tourney_date > datetime.datetime(1997, 12, 31)) &
                 (matches.tourney_date < datetime.datetime(2019, 1, 14))]


matches = matches[matches.tourney_date < datetime.datetime(2019, 1, 14)]


### Clean String Columns 

Let's standarize data that is non-numeric by stripping out leading/trailing spaces and converting to lowercase

We will also remove any special characters

In [4]:
# keys for the data are player names and tournament names - these are strings
# we also have come categorical columns - ie loser_hand, winner_hand, surface, tourney_level, 
# let's convert any non-numerical column data into lower case and strip
# we will also remove any special characters and accents

import unicodedata
import re

# first let's print one of the columns
print("Before lowering...")
print(matches[:5].loser_hand)

lower_columns = [col for col, dt in matches.dtypes.items() if dt == np.object]
for col in lower_columns:
    print(f'cleaning col {col}')
    matches[col] = matches[col].str.strip()
    matches[col] = matches[col].str.lower()
    matches[col] = matches[col].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    # we won't do this for score because they should look like 6-3 6-7(7)
    if col not in ["score", "tourney_id"]:
        matches[col] = matches[col].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x, flags=re.I | re.A))

# check to make sure we've done this correctly
print("After lowering...")
matches[:5].loser_hand

Before lowering...
0    R
1    R
2    R
3    R
4    R
Name: loser_hand, dtype: object
cleaning col tourney_id
cleaning col tourney_name
cleaning col surface
cleaning col tourney_level
cleaning col winner_name
cleaning col winner_hand
cleaning col winner_ioc
cleaning col loser_name
cleaning col loser_hand
cleaning col loser_ioc
cleaning col score
cleaning col round
After lowering...


0    r
1    r
2    r
3    r
4    r
Name: loser_hand, dtype: object

# Cleaning Scores

We will most likely break score into # of sets won by each user and # of games won by each user as feature

The current score has tiebreak points - since we are not going all the way down to point level, we can strip this out

In [5]:
import re

# first let's get some scores with tiebreak so we can test to make sure we did this correctly
tiebreaks = matches[matches.score.str.contains(r"\)")]
print(tiebreaks["score"].head(10))
tb_indexes = tiebreaks.index
tb_indexes

# strip out tiebreak scores
matches.loc[matches.score.str.contains("\)"), "score"] = matches[matches.score.str.contains("\)")].score.apply(lambda x: re.sub(r'\(\d+\)', '', x))





12    6-3 6-7(5) 6-3
15        7-6(5) 7-5
19    6-7(8) 6-2 6-4
21    4-6 7-6(1) 6-1
24    6-7(3) 6-1 7-5
29     7-6(5) 7-6(4)
30    3-6 6-3 7-6(4)
34    6-7(5) 6-4 6-4
36        6-2 7-6(3)
40        7-6(5) 6-4
Name: score, dtype: object


In [6]:
print(matches.iloc[tb_indexes[:10]]["score"])

# verify if there are any other tiebreak scores left
len(matches[matches.score.str.contains(r"\)")]["score"])


12    6-3 6-7 6-3
15        7-6 7-5
19    6-7 6-2 6-4
21    4-6 7-6 6-1
24    6-7 6-1 7-5
29        7-6 7-6
30    3-6 6-3 7-6
34    6-7 6-4 6-4
36        6-2 7-6
40        7-6 6-4
Name: score, dtype: object


0

In [7]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59293 entries, 0 to 65835
Data columns (total 45 columns):
tourney_id       59293 non-null object
tourney_name     59293 non-null object
surface          59293 non-null object
draw_size        0 non-null float64
tourney_level    59293 non-null object
tourney_date     59293 non-null datetime64[ns]
match_num        59293 non-null int64
winner_id        59293 non-null int64
winner_seed      27003 non-null float64
winner_name      59293 non-null object
winner_hand      59293 non-null object
winner_ht        57220 non-null float64
winner_ioc       59293 non-null object
winner_age       59293 non-null float64
loser_id         59293 non-null int64
loser_seed       14708 non-null float64
loser_name       59293 non-null object
loser_hand       59293 non-null object
loser_ht         55557 non-null float64
loser_ioc        59293 non-null object
loser_age        59293 non-null float64
score            59293 non-null object
best_of          59293 no

# Impute Missing Data

We are missing some matches's minutes (ie, length of the match). We can impute this - since matches length might depend on the tournament (ie, surface) and whether the tournament is best of 3 or 5 - we will use this inforamation to impute by using the mean of match minutes for that tournament


In [8]:
## Some matches are misssing minutes

In [9]:
# we are missing some matches's minutes (ie, length of the match)
# we can impute this - since matches length might depend on the tournament (ie, surface) 
# and whether the tournament is best of 3 or 5 - we will use this inforamation 
# to impute by using the mean of match minutes for that tournament
tids = {id for id in matches[matches.minutes.isnull()].tourney_id}
for tid in tids:
    matches.loc[(matches.minutes.isnull()) & (matches.tourney_id == tid), "minutes"] = \
             matches[(matches.minutes.notnull()) & (matches.tourney_id == tid)].minutes.mean()

In [10]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59293 entries, 0 to 65835
Data columns (total 45 columns):
tourney_id       59293 non-null object
tourney_name     59293 non-null object
surface          59293 non-null object
draw_size        0 non-null float64
tourney_level    59293 non-null object
tourney_date     59293 non-null datetime64[ns]
match_num        59293 non-null int64
winner_id        59293 non-null int64
winner_seed      27003 non-null float64
winner_name      59293 non-null object
winner_hand      59293 non-null object
winner_ht        57220 non-null float64
winner_ioc       59293 non-null object
winner_age       59293 non-null float64
loser_id         59293 non-null int64
loser_seed       14708 non-null float64
loser_name       59293 non-null object
loser_hand       59293 non-null object
loser_ht         55557 non-null float64
loser_ioc        59293 non-null object
loser_age        59293 non-null float64
score            59293 non-null object
best_of          59293 no

In [11]:
matches[matches.minutes.isnull()][["tourney_id", "minutes"]]

,tourney_id,minutes
3031,1998-604,NaN
3032,1998-604,NaN
3033,1998-604,NaN
3034,1998-604,NaN
3035,1998-604,NaN
...,...,...
56973,2015-605,NaN
56974,2015-605,NaN
56975,2015-605,NaN
56976,2015-605,NaN


Looks like there is still a subset of tournaments where we do not have any results. This is ok - minutes will be used as part of feature engineering. We will just have to skip over these records when we create features later.

Looks like '98 and '99 Grand Slam Cup did not record match minutes. This is a year end tournament: 
https://www.grandslamhistory.com/atp/grand-slam-cup-munich

In [12]:
matches[matches.tourney_id.isin(matches[matches.minutes.isnull()].tourney_id.tolist())]["tourney_name"].unique()

array(['grand slam cup', 'sydney olympics', 'athens olympics',
       's hertogenbosch', 'stuttgart', 'halle', 'queen s club',
       'nottingham', 'wimbledon', 'newport', 'bastad', 'bogota', 'umag',
       'atlanta', 'gstaad', 'hamburg', 'kitzbuhel', 'washington',
       'canada masters', 'cincinnati masters', 'winston salem', 'us open',
       'metz', 'st  petersburg', 'kuala lumpur', 'shenzhen', 'beijing',
       'tokyo', 'shanghai masters', 'moscow', 'stockholm', 'vienna',
       'basel', 'valencia', 'paris masters', 'tour finals'], dtype=object)

## Impute Height

We are missing some values for player height. This, we can impute by using the average height

In [13]:
# impute height with mean of players
matches.loc[matches.loser_ht.isnull(), 'loser_ht'] = matches.loser_ht.mean()
matches.loc[matches.winner_ht.isnull(), 'winner_ht'] = matches.winner_ht.mean()

In [14]:
matches.sample(5).T

,39205,11445,54957,54647,28221
tourney_id,2010-451,2001-410,2015-403,2015-495,2006-311
tourney_name,doha,monte carlo masters,miami masters,dubai,queen s club
surface,hard,clay,hard,hard,grass
draw_size,NaN,NaN,NaN,NaN,NaN
tourney_level,a,m,m,a,a
tourney_date,2010-01-04 00:00:00,2001-04-16 00:00:00,2015-03-25 00:00:00,2015-02-23 00:00:00,2006-06-12 00:00:00
match_num,3,56,84,31,13
winner_id,104312,102856,104545,103819,102179
winner_seed,NaN,2,22,2,NaN
winner_name,andreas seppi,gustavo kuerten,john isner,roger federer,antony dupuis


In [15]:
# Lastly, let's drop any rows where we don't have scores for the matches
matches = matches.dropna(axis=0, subset=["score", "minutes"])

### Let's see what else we are missing data for

In [16]:
def print_columns_with_missing_data(m: pd.DataFrame):
    print(m.columns[m.isnull().any()].tolist())
    
    
print_columns_with_missing_data(matches)

['draw_size', 'winner_seed', 'loser_seed', 'w_ace', 'w_df', 'w_svpt', 'w_1stin', 'w_1stwon', 'w_2ndwon', 'w_svgms', 'w_bpsaved', 'w_bpfaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stin', 'l_1stwon', 'l_2ndwon', 'l_svgms', 'l_bpsaved', 'l_bpfaced', 'winner_rank', 'loser_rank']


## Missing Rank

At first I thought missing rank might mean that the player

When you look closer, I don't recognize all players, but Tim Henman was an English player that was at the top of his game in 1998 and was in the top 10 according to his wikipedia page: https://en.wikipedia.org/wiki/Tim_Henman

So I think we should somehow impute this

In [17]:
matches[(matches.winner_rank.isnull())][["tourney_date", "tourney_id", "round", "tourney_name", "winner_name"]].head(10)

,tourney_date,tourney_id,round,tourney_name,winner_name
1872,1998-06-22,1998-540,r128,wimbledon,mikael tillstrom
2251,1998-07-20,1998-418,r64,washington,jimmy arias
2351,1998-07-27,1998-423,r32,los angeles,tim henman
2359,1998-07-27,1998-423,r16,los angeles,tim henman
2363,1998-07-27,1998-423,qf,los angeles,tim henman
2365,1998-07-27,1998-423,sf,los angeles,tim henman
2580,1998-08-17,1998-419,r64,indianapolis,wayne black
2600,1998-08-17,1998-419,r32,indianapolis,wayne black
2692,1998-08-24,1998-80,r32,boston,jiri novak
3078,1998-09-28,1998-327,r32,toulouse,lionel barthez


I think a reasonable way to to impute the data for that tournament. We will take the mean of the player rank in that round and impute into our missing ranks

In [18]:
def impute_missing_rank(matches: pd.DataFrame, missing_list: pd.DataFrame, name_col: str) -> pd.DataFrame:
    for index, row in missing_list.iterrows():
        current_round_matches = matches[(matches.tourney_id == row.tourney_id) & 
                                        (matches["round"] == row["round"]) & (matches[name_col].notnull())]
        if len(current_round_matches) == 0:
            print(f'Unable to find other matches in this round index: {index} tourney_id: {str(row["tourney_id"])} round: {row["round"]} column: {name_col}. Imputing with tournament ranking')
            matches.loc[index, name_col] = matches[(matches.tourney_id == row.tourney_id) &
                                                   (matches[name_col].notnull())][name_col].mean()
        else:
#             print(f'len: {len(current_round_matches)} current mean: {int(current_round_matches[name_col].mean())}')
            matches.loc[index, name_col] = int(current_round_matches[name_col].mean())

In [19]:
losers_missing_rank = matches[matches.loser_rank.isnull()][["tourney_id", "round", "loser_name"]]
print(f'Missing loser rank before imputing: {len(losers_missing_rank)}')
print(losers_missing_rank.head(5))
impute_missing_rank(matches, losers_missing_rank, "loser_rank")    

losers_missing_rank = matches[matches.loser_rank.isnull()][["tourney_id", "round", "loser_name"]]

print(f'Missing loser rank after imputing: {len(losers_missing_rank)}')

Missing loser rank before imputing: 146
     tourney_id round        loser_name
1014   1998-336   r32    wing luen wong
1017   1998-336   r32      peter nyborg
1935   1998-540   r64  mikael tillstrom
2195   1998-321   r64  nicolas lapentti
2269   1998-418   r32       jimmy arias
Unable to find other matches in this round index: 2366 tourney_id: 1998-423 round: f column: loser_rank. Imputing with tournament ranking
Unable to find other matches in this round index: 3905 tourney_id: 1999-495 round: f column: loser_rank. Imputing with tournament ranking
Missing loser rank after imputing: 0


In [20]:
winner_missing_ranks = matches[matches.winner_rank.isnull()][["tourney_id", "round", "winner_name"]]
print(f'Missing winner rank before imputing: {len(winner_missing_ranks)}')

impute_missing_rank(matches, winner_missing_ranks, "winner_rank")    

winner_missing_ranks = matches[matches.winner_rank.isnull()][["tourney_id", "round", "winner_name"]]
print(f'Missing winner rank after imputing: {len(winner_missing_ranks)}')

Missing winner rank before imputing: 46
Missing winner rank after imputing: 0


## Impute Draw Size

For single elimination tournaments - we can impute this by the first round

In [21]:
# let's look at some some random samples before we impute
rounds = ['r128', 'r64', 'r32', 'r16']
missing_draw_indexes = matches[matches["round"].isin(rounds)].sample(10, random_state = 1).index
print(missing_draw_indexes)
matches.loc[missing_draw_indexes][["tourney_name", "round", "draw_size"]]

Int64Index([61139, 33490, 13756, 62572, 35990, 47388, 20032, 23183, 16387,
            149],
           dtype='int64')


,tourney_name,round,draw_size
61139,istanbul,r32,NaN
33490,rotterdam,r32,NaN
13756,australian open,r128,NaN
62572,shanghai masters,r32,NaN
35990,st petersburg,r16,NaN
47388,washington,r32,NaN
20032,adelaide,r16,NaN
23183,stockholm,r16,NaN
16387,hong kong,r32,NaN
149,australian open,r128,NaN


In [22]:
# impute draw size for single elimination tournament
import re

# we are going to impute in reverse order so that we impute with max
for r in rounds:
    size = int(re.sub("r", "", r))
    # impute draw_size for the entire tournament
    tids = np.unique(matches[(matches["round"] == r) & (matches.draw_size.isnull())][["tourney_id"]])
    matches.loc[matches["tourney_id"].isin(tids), "draw_size"] = size
    
# let's look at the same random samples and see if they look correct
# round should be < then draw_size
matches.loc[missing_draw_indexes][["tourney_name", "round", "draw_size"]]
    

,tourney_name,round,draw_size
61139,istanbul,r32,32.0
33490,rotterdam,r32,32.0
13756,australian open,r128,128.0
62572,shanghai masters,r32,64.0
35990,st petersburg,r16,32.0
47388,washington,r32,32.0
20032,adelaide,r16,32.0
23183,stockholm,r16,32.0
16387,hong kong,r32,32.0
149,australian open,r128,128.0


Let's now look at Round Robin tournaments (rr)

Background - these tournaments generally are divided into pools, then winner of the pools go into a single elimination tournament of 4 at the end

In order to calculate this, we will have to look at how many players were in each tournament (winners and losers)

In [23]:
def get_draw_size(matches: pd.DataFrame, yti: str):
    tourney_matches = matches[matches.tourney_id == yti]
    pids = np.unique(np.append(tourney_matches.winner_id, tourney_matches.loser_id))
    matches.loc[matches.tourney_id == yti, "draw_size"] = len(pids)
    
    
for i in matches[matches["round"] == 'rr']["tourney_id"].unique():
    get_draw_size(matches, i)
    
print(matches[matches["round"] == 'rr'].sample(10, random_state=1)[["tourney_name", "round", "draw_size"]])

# check to see if there are any more tournaments without draw_size
print(f'Tournaments without draw_size left: {len(matches[matches.draw_size.isnull()].tourney_id)}')

       tourney_name round  draw_size
11800    dusseldorf    rr       17.0
54022   tour finals    rr        9.0
34404    dusseldorf    rr       20.0
43493    dusseldorf    rr       20.0
24700    dusseldorf    rr       19.0
30382  buenos aires    rr       32.0
42155   tour finals    rr        8.0
5038     dusseldorf    rr       18.0
5044     dusseldorf    rr       18.0
48176   tour finals    rr        8.0
Tournaments without draw_size left: 0


## Player Seed

We are missing quite a lot of player seeds but we can derive this

Player seed is determined by the player's rank. Player with the highest rank in the tournament will get seed #1, etc

In [24]:
# We will impute both loser_seed and winner_seed

# let's get a list of tournaments where we are missing either loser_seed or winner_seed
tids_seed = np.unique(matches[(matches.loser_seed.isnull()) | (matches.winner_seed.isnull())]["tourney_id"])

# get list of all players and their anks for the tournaments
for tid in tids_seed:
    tourney_matches = matches[matches.tourney_id == tid]
    winners = pd.DataFrame(tourney_matches[["winner_id", "winner_rank"]]).rename({"winner_id": "player_id", "winner_rank": "rank"}, axis=1)
    losers = pd.DataFrame(tourney_matches[["loser_id", "loser_rank"]]).rename({"loser_id": "player_id", "loser_rank": "rank"}, axis=1)

    # let's seed all players in the tournament
    players = winners.append(losers, ignore_index=True).drop_duplicates().sort_values("rank")
    counter = 1
    for index, player in players.iterrows():
        players.loc[index, "seed"] = counter
        counter += 1

    # print(players)

    for index, match in tourney_matches.iterrows():
        if pd.isnull(match.winner_seed):
            seed = players[players.player_id == match.winner_id]["seed"].values[0]
    #         print(f'index {index} winner_id {match.winner_id} winner seed {seed}')
            matches.loc[index, 'winner_seed'] = players[players.player_id == match.winner_id]["seed"].values[0]
        if pd.isnull(match.loser_seed):
            seed = players[players.player_id == match.loser_id]["seed"].values[0]
    #         print(f'index {index} loser_id {match.loser_id} loser seed {seed}')
            matches.loc[index, 'loser_seed'] = seed
        
# randomly pick a tournament and see how we did
matches[matches.tourney_id == tids_seed[10]][["winner_rank", "winner_seed", "loser_rank", "loser_seed"]]

,winner_rank,winner_seed,loser_rank,loser_seed
2187,121.0,46.0,151.0,48.0
2188,97.0,42.0,73.0,33.0
2189,81.0,36.0,36.0,18.0
2190,37.0,19.0,102.0,43.0
2191,53.0,28.0,94.0,41.0
2192,82.0,37.0,75.0,34.0
2193,42.0,24.0,86.0,39.0
2194,38.0,20.0,41.0,23.0
2195,65.0,32.0,79.0,35.0
2196,112.0,44.0,34.0,16.0


In [25]:
matches[matches.loser_rank.isnull()][["tourney_name", "tourney_id", "draw_size", "loser_name"]]

,tourney_name,tourney_id,draw_size,loser_name


# Done

OK. Looks like we have cleaned up and inputed as much data as we can as of this point

For this round, we will not be using any of the match statistics so we will not drop rows with empty data for these columns.

However, we will drop the two rows for loser_rank and then save to file so we can move on to feature engineering

In [26]:
print_columns_with_missing_data(matches)

['w_ace', 'w_df', 'w_svpt', 'w_1stin', 'w_1stwon', 'w_2ndwon', 'w_svgms', 'w_bpsaved', 'w_bpfaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stin', 'l_1stwon', 'l_2ndwon', 'l_svgms', 'l_bpsaved', 'l_bpfaced']


### Save our data

In [29]:
matches.to_csv(f'{OUTFILE}', index=False)